# Installation
To install crewAI, you need to have Python >=3.10 and <=3.13 installed on your system:

In [ ]:
# Install the mains crewAI package
!pip install crewai

# Install the main crewAI package and the tools package
# that includes a series of helpful tools for your agents
!pip install 'crewai[tools]'

# Getting Started
https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/
## Step 1: Assemble Your Agents
에이전트의 roles, backstories, 그리고 상세 모드와 메모리 사용 같은 향상된 기능을 정의. 이러한 요소들은 깊이를 더해주며, 팀 내에서의 작업 수행과 상호 작용을 안내함.

- **Attributes**
  
    - ```role```: Agent의 역할 정의
    - ```goal```: Agent의 목표 지정
    - ```backstory```: Agent의 backstory 제공.
    - ```llm```: Agent가 사용하는 LLM모델. default값으로 환경 변수 "OPENAI_MODEL_NAME"에서 정의된 GPT-4 모델을 사용
    - ```function_calling_llm``` ***Optional***: ReAct(Reasoning+Action) crewAI 에이전트를 함수 호출 에이전트로 변환
    - ```max_iter```: Agent가 작업을 실행하는 최대 반복 횟수. Default is 15.
    - ```memory```: Agent가 실행 중에 정보를 보존할 수 있도록 함. Default is ```False```.
    - ```max_rpm```: Agent가 요청하는 분당 최대 요청 수. Optional.
    - ```verbose```: Agent 실행의 상세 로깅 활성화. Default is ```False```.
    - ```allow_delegation```: Agent가 다른 Agent에게 작업을 위임할 수 있도록 함, default is ```True```.
    - ```tools```: task실행을 위해 Agent에게 제공된 tool을 구체화 함. Optional.
    - ```step_callback```: 각 단계 후에 실행할 callback 함수 제공. Optional.
    - ```cache```: tool 사용에 대해 cache할 지 결정. Default is ```True```.

In [ ]:
import os
os.environ["SERPER_API_KEY"] = "***************************"  # serper.dev API key
os.environ["OPENAI_API_KEY"] = "sk-***************************"
os.environ["OPENAI_MODEL_NAME"]="gpt-4o"

In [ ]:
from crewai import Agent
from crewai_tools import SerperDevTool
search_tool = SerperDevTool()

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
    role='Senior Researcher',
    goal='Uncover groundbreaking technologies in {topic}',
    verbose=True,
    memory=True,
    backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
    ),
    tools=[search_tool],
    allow_delegation=True
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False
)

## Step 2: Define the Tasks
Agent의 구체적인 목표 설정. 비동기 실행, output customization 등의 새로 나온 기능을 포함하여 task는 agent의 역할에 대한 target 접근을 보장함

In [ ]:
from crewai import Task

# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

## Step 3: Form the Crew
Agent들을 하나의 크루로 결합하여 작업을 수행할 워크플로우 프로세스를 설정함. 옵션과 성능 최적화를 위한 추가 설정을 포함하여 향상된 상호 작용을 도울 수 있음.

In [ ]:
from crewai import Crew, Process

# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True
)

## Step 4: Kick It Off
Crew() 함수 내에 사용 목적에 맞는 Agent와 Task를 설정. Agent들은 서로 협업하여 사용자가 지정한 task에 맞게 서로 보간되어 최적의 결과를 답변할 것임.

In [ ]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
print(result)

<hr>

# markdown validator

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
from getpass import getpass
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from textwrap import dedent

os.environ["OPENAI_API_KEY"] = 'sk-*******************************'

## 마크다운 validator 설치
https://github.com/joaomdmoura/crewAI-examples/tree/main/markdown_validator#introduction

1. .env 파일 생성 후 환경변수 설정
   ```shell
   # Using OpenAI's API
   OPENAI_API_KEY="sk-****************************************"
   MODEL_NAME="gpt-4o"
   ```
2. Install Dependencies
   ```shell
   poetry install --no-root
   ```

## syntax 검사 agent와 마크다운 작성자 agent 배치

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from crewai import Agent
from markdown_validator.MarkdownTools import markdown_validation_tool

# 마크다운 문법 검사 agent
review_agent = Agent(
    role=dedent((
        """
        'Requirements Manager'
        """)), # Think of this as the job title
    backstory=dedent((
        """
        You are an expert synthetic biology analyst and software QA specialist. You provide high quality, thorough, insightful and actionable feedback via detailed list of changes and actionable tasks.
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        Provide a detailed list of the markdown linting results. Give a summary with actionable tasks to address the validation results. Write your response as if you were handing it to a developer to fix the issues. DO NOT provide examples of how to fix the issues or recommend other tools to use.
        """)), # This is the goal that the agent is trying to achieve
    tools=[markdown_validation_tool],
    allow_delegation=False,
    # verbose=True,
    memory=True,
    # ↑ Whether the agent execution should be in verbose mode
    max_iter=3,
    # ↑ maximum number of iterations the agent can perform before being forced to give its best answer (generate the output)
    max_rpm=100, # This is the maximum number of requests per minute that the agent can make to the language model
    llm=ChatOpenAI(model_name="gpt-4o", top_p=0.8)
)

# 마크다운 작성자
writer_agent = Agent(
    role=dedent((
        """
        Writer
        """)), # Think of this as the job title
    backstory=dedent((
        """
        You share the markdown file received as input with the agent that recognizes the error to correct it.
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        Resume grammatical errors in the full markdown doc form. Return full content, not just content with grammatical errors. DO NOT provide examples of how to fix the issues or recommend other tools to use.
        """)), # This is the goal that the agent is trying to achieve
    
    allow_delegation=False,
    # verbose=True,
    # ↑ Whether the agent execution should be in verbose mode
    max_iter=3,
    # ↑ maximum number of iterations the agent can perform before being forced to give its best answer (generate the output)
    max_rpm=100, # This is the maximum number of requests per minute that the agent can make to the language model
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8)
)

In [ ]:
!pip install StringIO

## 마크다운 파일 불러오기

In [ ]:
import markdown

with open('markdown_validator/protein2.md', 'r', encoding='utf-8') as file:
    markdown_content = file.read()

## 각 agent들의 task 부여

In [ ]:
import datetime
from crewai import Task

# Define Tasks Using Crew Tools
syntax_review_task = Task(description=f'''
        Use the markdown_validation_tool to review the file(s) at this path: "{"markdown_validator/protein2.md"}"
        
        Be sure to pass only the file path to the markdown_validation_tool.
        Use the following format to call the markdown_validation_tool:
        Do I need to use a tool? Yes
        Action: markdown_validation_tool
        Action Input: "{"markdown_validator/protein2.md"}"

        Get the validation results from the tool 
        and then summarize it into a list of changes
        the developer should make to the document.
        DO NOT recommend ways to update the document.
        DO NOT change any of the content of the document or
        add content to it. It is critical to your task to
        only respond with a list of changes.
        
        If you already know the answer or if you do not need 
        to use a tool, return it as your Final Answer.''',
        expected_output="""
        Line numbers and content with grammatical errors.
        """,
        agent=review_agent)

writer_task = Task(description=dedent((f"""
        Please correct the syntax issues found in "{markdown_content}" and provide them
        """)),
        expected_output=dedent(("""
            Markdown file that corrects only grammatical errors without modifying the content.
            """)),
        context=[syntax_review_task],
        # ↑ specify which task's output should be used as context for subsequent tasks
    max_rpm=100,
        output_file=f'result_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md')

## 마크다운 검사를 위한 Crew 생성

In [ ]:
from crewai import Crew, Process

def main():
    # Instantiate your crew with a sequential process
    md_crew = Crew(
        agents=[review_agent, writer_agent],
        tasks=[syntax_review_task, writer_task],
        # verbose=2,  # You can set it to 1 or 2 to different logging levels
        # ↑ indicates the verbosity level for logging during execution.
        process=Process.sequential
        # ↑ the process flow that the crew will follow (e.g., sequential, hierarchical).
    )

    result = md_crew.kickoff()
    print("\n\n########################")
    print("## Here is your markdown crew run result:")
    print("########################\n")
    print(result)

if __name__ == "__main__":
  result = main()

<hr>

# 마크다운 RAG 및 생성

In [ ]:
from crewai import Agent, Task
from crewai_tools import MDXSearchTool

# Initialize the tool with a specific MDX file path for an exclusive search within that document
md_tool = MDXSearchTool()
# mdx=['markdown_validator/020_multi_module_to_plasmid.md', 'markdown_validator/protein2.md']

md_collector = Agent(
    role='a Ph.D. in synthetic biology',
    goal='Design a synthetic biology experiment based on a given {input1}, {input2} and understand the markdown structure',
    tools=[md_tool],
    verbose=True,
    memory=True,
    backstory=(
        "You understand the overall experimental design stage of synthetic biology."
        "You understand the concept of each task, workflow, and unit process and know how to see the markdown structure according to this criterion."
        "The experiment name is header2 (##), the unit process in the experiment is header3 (####), and the equipment, reagents, methods, results, and consumables required to construct the unit process are composed of header4 (#####)."
    ),
    max_rpm=100,
    allow_delegation=False
)
fetch_md_task = Task(
    description=(
        "Bring the synthetic biology experiment stage related to the {input1}, {input2}"
        "Understand the markdown structure according to each step and notice that task, workflow, and unit process are divided according to the header part of this markdown"
    ),
    expected_output='Raw data ready for processing.',
    agent=md_collector
)

In [ ]:
import datetime
md_writer = Agent(
    role='a synthetic biology experimental designer',
    goal='Transforms the information and structure of raw experimental data into a structured format for the {input3} experiment.',
    backstory=(
        "Create an experimental design in a markdown format"
        "Your work ensures that the information is not only accurate but also pleasing to read."
        "The experiment name is header2 (##), the unit process in the experiment is header3 (####), and the equipment, reagents, methods, results, and consumables required to construct the unit process are composed of header4 (####)."
    ),
    max_rpm=100,
    allow_delegation=False
)

md_writer_task = Task(
    description=(
        "Obtain the structure and information of the raw data from md_collector and configure {input3} in a structured markdown format."
        "Design a new synthetic biology experiment by identifying the structure of each header and content"
    ),
    expected_output='Organized content ready for Markdown formatting.',
    agent=md_writer,
    output_file=f'result_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md'
)

In [ ]:
from crewai import Crew, Process 

# 정의된 에이전트 및 작업을 사용하여 크루 초기화
md_crew = Crew( 
    name= 'markdown_writer' , 
    agents=[md_collector, md_writer], 
    tasks=[fetch_md_task, md_writer_task], 
    process= Process.sequential,   # 작업이 추가된 순서대로 실행되는지 확인합니다.
    verbose= True , 
    memory= True , 
    cache= False , 
    max_rpm= 100
) 
if __name__ == '__main__' : 
    result = md_crew.kickoff(inputs={
        'input1' : 'markdown_validator/020_multi_module_to_plasmid.md',
        'input2' : 'markdown_validator/protein2.md',
        'input3' : 'MPH Single Directly Mutation'}) 
    print ( "Crew Execution Result:" , result)